In [1]:
!pip install keras

    100% |████████████████████████████████| 348kB 2.6MB/s ta 0:00:01
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
ratings= pd.read_csv('/content/datalab/matrix_factorization_keras.csv')

In [4]:
# Extract the unique users
users = ratings.User.unique()

In [5]:
# Extract the unique movies
articles = ratings.Movies.unique()

In [6]:
# Index each user and article
userid2idx = {o:i for i,o in enumerate(users)}
articlesid2idx = {o:i for i,o in enumerate(articles)}

In [7]:
# Apply the index created to the original dataset
ratings.Movies = ratings.Movies.apply(lambda x: articlesid2idx[x])
ratings.User = ratings.User.apply(lambda x: userid2idx[x])

In [8]:
# Extract the number of unique users and articles
n_users = ratings.User.nunique()
n_articles = ratings.Movies.nunique()

In [9]:
# Define the error metric
import keras.backend as K
def rmse(y_true,y_pred):
    score = K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
    return score

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
# Import relevant packages
from keras.layers import Input, Embedding, Dense, Dropout, merge, Flatten
from keras.models import Model

In [11]:
def embedding_input(name,n_in,n_out):
  inp = Input(shape=(1,),dtype='int64',name=name)
  return inp, Embedding(n_in,n_out,input_length=1)(inp)

In [12]:
n_factors = 2
user_in, u = embedding_input('user_in', n_users, n_factors)
article_in, a = embedding_input('article_in', n_articles, n_factors)

In [13]:
# Initialize the dot product between user matrix and movie matrix
x = merge([u,a],mode='dot')
x=Flatten()(x)

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app
/usr/local/envs/py3env/lib/python3.5/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [15]:
# Initialize the model specification
from keras import optimizers
model = Model([user_in,article_in],x)
sgd = optimizers.SGD(lr=0.01)
model.compile(sgd,loss='mse',metrics=[rmse])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_in (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
article_in (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 2)         10          user_in[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 2)         8           article_in[0][0]                 
__________________________________________________________________________________________________
merge_1 (M

In [ ]:
# Fit the model by specifying inputs and output
model.fit([ratings.User,ratings.Movies], ratings.Actual, nb_epoch=1000, batch_size=13)

13/13 [==============================] - 0s 468us/step - loss: 0.0467 - rmse: 0.2161
Epoch 721/1000
13/13 [==============================] - 0s 283us/step - loss: 0.0463 - rmse: 0.2152
Epoch 722/1000
13/13 [==============================] - 0s 310us/step - loss: 0.0460 - rmse: 0.2144
Epoch 723/1000
13/13 [==============================] - 0s 284us/step - loss: 0.0456 - rmse: 0.2135
Epoch 724/1000
13/13 [==============================] - 0s 277us/step - loss: 0.0452 - rmse: 0.2127
Epoch 725/1000
13/13 [==============================] - 0s 277us/step - loss: 0.0449 - rmse: 0.2119
Epoch 726/1000
13/13 [==============================] - 0s 264us/step - loss: 0.0446 - rmse: 0.2111
Epoch 727/1000
13/13 [==============================] - 0s 264us/step - loss: 0.0442 - rmse: 0.2103
Epoch 728/1000
13/13 [==============================] - 0s 271us/step - loss: 0.0439 - rmse: 0.2095
Epoch 729/1000
13/13 [==============================] - 0s 317us/step - loss: 0.0435 - rmse: 0.2087
Epoch 730/1000


In [18]:
# User matrix
model.get_weights()[0]

array([[ 1.3606772,  1.6446526],
       [ 1.0029647,  1.3566955],
       [-1.4094337,  1.6923367],
       [-1.0425398,  1.4308459],
       [-1.2908196,  1.4010738]], dtype=float32)

In [19]:
# Movie matrix
model.get_weights()[1]

array([[-1.3724959,  1.7509296],
       [ 1.5061257,  1.8171026],
       [ 0.6696424,  1.235024 ],
       [-1.7306372,  1.8775703]], dtype=float32)